In [7]:
import json
import numpy as np

In [29]:
with open("centerofmasses.json", "r") as f:
    centerofmasses = json.load(f)

dt = 1/30

pos_vel_acc = {}

all_positions = []
all_velocities = []
all_accelerations = []
new_all_positions = []

for key in centerofmasses:
    all_positions.append(centerofmasses[key])
min_x=-6.755555555555555
max_x=6.755555555555555
min_y=-3.8
max_y=3.8
left_crop_percent = 0.12763466042154567 
right_crop_percent = 0.8723653395784543
top_crop_percent = 0.0020833333333333333
bottom_crop_percent = 0.9958333333333333

def video_coordinate_to_scene(cx, cy):
    x_length = max_x - min_x
    y_length = max_y - min_y
    video_x_min = min_x + left_crop_percent * x_length
    video_x_max = min_x + right_crop_percent * x_length
    video_y_min = min_y + top_crop_percent * y_length
    video_y_max = min_y + bottom_crop_percent * y_length
    x_normalized = cx / 960 * (video_x_max - video_x_min) + video_x_min
    y_normalized = (720 - cy) / 720 * (video_y_max - video_y_min) + video_y_min
    return x_normalized, y_normalized

def normalize_and_add_z(pos):
    x = pos[0]
    y = pos[1]
    x, y = video_coordinate_to_scene(x, y)
    return [x, y, 0]

print(len(all_positions))
for index in range(len(all_positions)):
    current_position = all_positions[index]
    current_position = normalize_and_add_z(current_position)
    # add some sine wave to z
    current_position[2] += 0.2*np.sin(2*np.pi*index/30)
    next_position = all_positions[index+1] if index < len(all_positions)-1 else all_positions[index]
    next_position = normalize_and_add_z(next_position)
    last_position = all_positions[index-1] if index > 0 else all_positions[index]
    last_position = normalize_and_add_z(last_position)
    current_position = np.array(current_position)
    next_position = np.array(next_position)
    last_position = np.array(last_position)

    new_all_positions.append(current_position.tolist())
    velocity = (next_position - current_position) / dt
    all_velocities.append(velocity.tolist())
    acceleration = (next_position - 2*current_position + last_position) / (dt**2)
    all_accelerations.append(acceleration.tolist())

for index in range(len(all_positions)):
    pos_vel_acc[index] = {
        "position": new_all_positions[index],
        "velocity": all_velocities[index],
        "acceleration": all_accelerations[index]
    }

print(pos_vel_acc[50])

with open("pos_vel_acc_z_sin.json", "w") as f:
    json.dump(pos_vel_acc, f, indent=4)
    


6572
{'position': [-1.5498436299810634, -0.1509510711774773, -0.17320508075688776], 'velocity': [-1.0919739561216923, -2.2945472753448026, 5.196152422706633], 'acceleration': [158.23373474392835, -29.548827678713696, 311.769145362398]}


In [ ]:
# I need a function to calculate electric field as a function of x, y, and t: E(x, y, t)

pos_vel_acc = json.load(open("pos_vel_acc.json", "r"))
c_light = 299792458
k = 8.9875517873681764e9
q_charge = 1.60217662e-19
def E(x, y, t):
    # Electric field due to moving point charge, following Griffiths eq 10.72
    distance = np.sqrt(x**2 + y**2)
    retarded_time = t - distance / c_light
    # interpolate between the two closest points
    ceil_index = int(np.ceil(retarded_time / dt))
    floor_index = int(np.floor(retarded_time / dt))
    if ceil_index == floor_index:
        position = pos_vel_acc[floor_index]["position"]
        velocity = pos_vel_acc[floor_index]["velocity"]
        acceleration = pos_vel_acc[floor_index]["acceleration"]
    else:
        position = 1/2 * (pos_vel_acc[floor_index]["position"] + pos_vel_acc[ceil_index]["position"])
        velocity = 1/2 * (pos_vel_acc[floor_index]["velocity"] + pos_vel_acc[ceil_index]["velocity"])
        acceleration = 1/2 * (pos_vel_acc[floor_index]["acceleration"] + pos_vel_acc[ceil_index]["acceleration"])
    # add z component of zero to all
    position.append(0)
    velocity.append(0)
    acceleration.append(0)
    w = np.array(position)
    v = np.array(velocity)
    a = np.array(acceleration)
    curly_r = np.array([x, y, 0]) - w
    curly_r_hat = curly_r / np.linalg.norm(curly_r)
    u = c_light * curly_r_hat - v
    curly_r_norm = np.linalg.norm(curly_r)
    # divide by dot product of curly r and u cubed
    constant_out_front = k * q_charge * curly_r_norm / np.dot(curly_r, u)**3
    v_magnitude = np.linalg.norm(v)
    vector_term = (c_light**2 - v_magnitude**2) * u + np.cross(curly_r, np.cross(u, a))
    return constant_out_front * vector_term

